In [2]:
import pandas as pd
from embeds import multi_inner_align, standardize
import sys
sys.path.append('..')
from rca.rca import run_rca

In [5]:
ft_baseline = pd.read_csv('../../data/embeds/fastText_CommonCrawl.csv', index_col=0)
Llama_3_1B = pd.read_csv('../../data/llms/Llama-3.2-1B.csv', index_col=0)

# Comparing
to_compare = {
    'ft_baseline': ft_baseline,
    'Llama_3.2_1B': Llama_3_1B
}

# Aligning for fair comparison
to_compare = dict(zip(to_compare.keys(), multi_inner_align(to_compare.values())))

# Standardizing
to_compare = {name: standardize(embed) for name, embed in to_compare.items()}

# Loading norm data
norms = pd.read_csv('../../data/psychNorms/psychNorms_processed.zip', index_col=0, low_memory=False, compression='zip')
norms_meta = pd.read_csv('../../data/psychNorms/psychNorms_metadata_processed.csv', index_col='norm')


[embed.shape for embed in to_compare.values()]

[(44450, 300), (44450, 2048)]

In [ ]:
results = run_rca(to_compare, norms, norms_meta, n_jobs=10)
results

  0%|          | 0/2 [00:00<?, ?it/s]

ft_baseline:   0%|          | 0/291 [00:00<?, ?it/s]

In [ ]:
# Adding norm category
results['norm_category'] = (
    results['norm']
    .apply(lambda norm: norms_meta.loc[norm]['category'])
    .replace({'_': ' '}, regex=True)
)

results_avg = (
    results[['norm_category', 'embed', 'r2_mean']]
    .groupby(['norm_category', 'embed'], as_index=False).median()
    .dropna()
)

results_avg_piv = results_avg.pivot(columns='embed', index='norm_category', values='r2_mean')
results_avg_piv.round(2)

In [ ]:
# Finding the top-performing fmri_text_denoise
sorted_overall = results_avg_piv.mean().sort_values(ascending=False)
sorted_overall

In [ ]:
## Saving
top_performer = None
top_performer.to_csv('../../data/embeds/Llama_X_XB.csv')